In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/binggas/public-dataset/main/bank%2Bmarketing/bank-additional/bank-additional-full-alt.csv")
len(df)

41188

In [5]:
# Add customer ID

df["customer_id"] = df["age"].astype(str) + '_' + df["job"].astype(str) + '_' + df["marital"].astype(str) + '_' + \
                      df["education"].astype(str) + '_' + df["housing"].astype(str) + '_' + df["loan"]
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,customer_id
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,56_housemaid_married_basic.4y_no_no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,57_services_married_high.school_no_no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,37_services_married_high.school_yes_no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,40_admin._married_basic.6y_no_no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,56_services_married_high.school_no_yes


In [24]:
len(df['customer_id'].unique())

10469

In [6]:
# Find the number of row by customer_id and campaign.
campaign_count = pd.DataFrame(df.groupby(["customer_id", "campaign"]).size())

# Rename the result column into `contact_per_campaign`
campaign_count.rename(columns={0:"contact_per_campaign"}, inplace=True)

# Show the first 5 row
campaign_count.head()

contact_per_campaign
customer_id                                campaign                      
17_student_single_basic.9y_unknown_unknown 2                            1
17_student_single_basic.9y_yes_no          2                            1
                                           3                            1
17_student_single_unknown_no_yes           1                            1
17_student_single_unknown_yes_no           3                            1

In [7]:
# Find the number of row by customer_id, campaign, and y.
campaign_respond_count = pd.DataFrame(df.groupby(["customer_id", "campaign", "y"]).size())

# Rename the result column into `count_respond`
campaign_respond_count.rename(columns={0:"count_respond"}, inplace=True)

# Only get the result where y = yes
campaign_respond_count = campaign_respond_count.loc[(slice(None), slice(None), "yes"), :]

# Show the first 5 row
campaign_respond_count.head()

,,,count_respond
customer_id,campaign,y,
17_student_single_basic.9y_unknown_unknown,2,yes,1
17_student_single_unknown_no_yes,1,yes,1
18_student_single_basic.4y_yes_no,1,yes,1
18_student_single_basic.6y_no_yes,2,yes,1
18_student_single_basic.9y_no_no,2,yes,1


In [8]:
# Merge campaign_respond_count with campaign_count
campaign_response = campaign_respond_count.merge(campaign_count, left_index=True, right_index=True)

# Calculate response_rate
campaign_response["response_rate"] = campaign_response["count_respond"] / campaign_response["contact_per_campaign"]

# Show the first 5 row
campaign_response.head()

,,,count_respond,contact_per_campaign,response_rate
customer_id,campaign,y,,,
17_student_single_basic.9y_unknown_unknown,2,yes,1,1,1.0
17_student_single_unknown_no_yes,1,yes,1,1,1.0
18_student_single_basic.4y_yes_no,1,yes,1,2,0.5
18_student_single_basic.6y_no_yes,2,yes,1,1,1.0
18_student_single_basic.9y_no_no,2,yes,1,1,1.0


In [9]:
# Aggregate data per customer_id
customer_agg = df.groupby("customer_id").agg({"customer_id":"count",
                                              "duration":["sum","mean"],
                                              "campaign":"nunique"
                                              })

# If multiindex, use the code below to flatten the columns
customer_agg.columns = customer_agg.columns.get_level_values(1)

# Rename the columns appropriately
customer_agg.rename(columns={"count": "contact_frequency",
                             "sum": "sum_duration_of_contact",
                             "mean": "avg_duration_of_contact",
                             "nunique": "campaign_engagement_cnt"
                             }, inplace=True)

# Show the first 5 row
customer_agg.head()

,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt
customer_id,,,,
17_student_single_basic.9y_unknown_unknown,1,498,498.0,1
17_student_single_basic.9y_yes_no,2,274,137.0,2
17_student_single_unknown_no_yes,1,896,896.0,1
17_student_single_unknown_yes_no,1,432,432.0,1
18_student_single_basic.4y_no_no,1,108,108.0,1


In [10]:
# Label for `1 Campaign`
customer_agg.loc[customer_agg["campaign_engagement_cnt"] == 1,"campaign_engagement_level"] = "1 Campaign"

# Label for `2-4 Campaign`
customer_agg.loc[(customer_agg["campaign_engagement_cnt"] > 1) & (customer_agg["campaign_engagement_cnt"] < 5),"campaign_engagement_level"] = "2-4 Campaigns"

# Label for `5+ Campaign`
customer_agg.loc[customer_agg["campaign_engagement_cnt"] >= 5,"campaign_engagement_level"] = "5+ Campaigns"

# Check if the logic works by showing the first 5 row
customer_agg.head()

,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt,campaign_engagement_level
customer_id,,,,,
17_student_single_basic.9y_unknown_unknown,1,498,498.0,1,1 Campaign
17_student_single_basic.9y_yes_no,2,274,137.0,2,2-4 Campaigns
17_student_single_unknown_no_yes,1,896,896.0,1,1 Campaign
17_student_single_unknown_yes_no,1,432,432.0,1,1 Campaign
18_student_single_basic.4y_no_no,1,108,108.0,1,1 Campaign


In [11]:
# Sanity check the result by aggregating row count for every `campaign_engagement_level` and `campaign_engagement_cnt`
customer_agg.groupby(["campaign_engagement_level", "campaign_engagement_cnt"]).size()

campaign_engagement_level  campaign_engagement_cnt
1 Campaign                 1                          5680
2-4 Campaigns              2                          2234
                           3                          1102
                           4                           630
5+ Campaigns               5                           362
                           6                           213
                           7                           138
                           8                            67
                           9                            24
                           10                           14
                           11                            2
                           12                            1
                           13                            2
dtype: int64

In [12]:
campaign_merge = campaign_response.merge(customer_agg, how="inner", left_index=True, right_index=True)
campaign_merge.head()

,,,count_respond,contact_per_campaign,response_rate,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt,campaign_engagement_level
customer_id,campaign,y,,,,,,,,
17_student_single_basic.9y_unknown_unknown,2,yes,1,1,1.0,1,498,498.0,1,1 Campaign
17_student_single_unknown_no_yes,1,yes,1,1,1.0,1,896,896.0,1,1 Campaign
18_student_single_basic.4y_yes_no,1,yes,1,2,0.5,2,548,274.0,1,1 Campaign
18_student_single_basic.6y_no_yes,2,yes,1,1,1.0,1,368,368.0,1,1 Campaign
18_student_single_basic.9y_no_no,2,yes,1,1,1.0,1,412,412.0,1,1 Campaign


In [13]:
# Defining function for 25th Percentile
def p25(x):
    return np.percentile(x,0.25)


# Defining function for 50th Percentile
def p50(x):
    return np.percentile(x,0.5)


# Defining function for 75th Percentile
def p75(x):
    return np.percentile(x,0.75)


# Defining function for 95th Percentile
def p95(x):
    return np.percentile(x,0.95)

In [14]:
# Calculate distribution statistics for response_rate per campaign_engagement_level
campaign_engagement_level = campaign_merge.groupby("campaign_engagement_level").agg({"response_rate": ["min", "mean", "max", p25, p50, p75, p95]})

# Flatten multiindex column
campaign_engagement_level.columns = campaign_engagement_level.columns.get_level_values(1)

# Reset index
campaign_engagement_level.reset_index(inplace=True)

# Show the result
campaign_engagement_level

,campaign_engagement_level,min,mean,max,p25,p50,p75,p95
0,1 Campaign,0.166667,0.880680,1.0,0.200000,0.250000,0.250000,0.250000
1,2-4 Campaigns,0.071429,0.661390,1.0,0.090909,0.100000,0.111111,0.111111
2,5+ Campaigns,0.038462,0.371066,1.0,0.047608,0.049976,0.050000,0.052632


In [15]:
# Melt the previous DataFrame and save it into a new DataFrame called campaign_engagement_melt

campaign_engagement_melt = pd.melt(campaign_engagement_level, id_vars=["campaign_engagement_level"], value_vars=["min", "mean", "max", "p25", "p50", "p75", "p95"])
campaign_engagement_melt.head(10)

,campaign_engagement_level,variable,value
0,1 Campaign,min,0.166667
1,2-4 Campaigns,min,0.071429
2,5+ Campaigns,min,0.038462
3,1 Campaign,mean,0.880680
4,2-4 Campaigns,mean,0.661390
5,5+ Campaigns,mean,0.371066
6,1 Campaign,max,1.000000
7,2-4 Campaigns,max,1.000000
8,5+ Campaigns,max,1.000000
9,1 Campaign,p25,0.200000


In [16]:
# copy campaign_merge into a new DataFrame called campaign_score
campaign_score = campaign_merge.copy()

# reset index
campaign_score.reset_index(inplace=True)

# show first 5 rows
campaign_score.head()

,customer_id,campaign,y,count_respond,contact_per_campaign,response_rate,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt,campaign_engagement_level
0,17_student_single_basic.9y_unknown_unknown,2,yes,1,1,1.0,1,498,498.0,1,1 Campaign
1,17_student_single_unknown_no_yes,1,yes,1,1,1.0,1,896,896.0,1,1 Campaign
2,18_student_single_basic.4y_yes_no,1,yes,1,2,0.5,2,548,274.0,1,1 Campaign
3,18_student_single_basic.6y_no_yes,2,yes,1,1,1.0,1,368,368.0,1,1 Campaign
4,18_student_single_basic.9y_no_no,2,yes,1,1,1.0,1,412,412.0,1,1 Campaign


In [18]:
# Get 50th percentile response_rate, group by customer_id. Save it into campaign_rate_per_customer
campaign_rate_per_customer = pd.DataFrame(campaign_score.groupby("customer_id")["response_rate"].agg(p50))

# Rename to result column to be campaign_rate_per_customer
campaign_rate_per_customer.rename(columns={"response_rate":"campaign_rate_per_customer"}, inplace=True)

# Merge campaign_score with campaign_rate_per_customer by customer_id
campaign_score = campaign_score.merge(campaign_rate_per_customer, how="left", on="customer_id")

# Show first 5 rows
campaign_score.head()

,customer_id,campaign,y,count_respond,contact_per_campaign,response_rate,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt,campaign_engagement_level,campaign_rate_per_customer
0,17_student_single_basic.9y_unknown_unknown,2,yes,1,1,1.0,1,498,498.0,1,1 Campaign,1.0
1,17_student_single_unknown_no_yes,1,yes,1,1,1.0,1,896,896.0,1,1 Campaign,1.0
2,18_student_single_basic.4y_yes_no,1,yes,1,2,0.5,2,548,274.0,1,1 Campaign,0.5
3,18_student_single_basic.6y_no_yes,2,yes,1,1,1.0,1,368,368.0,1,1 Campaign,1.0
4,18_student_single_basic.9y_no_no,2,yes,1,1,1.0,1,412,412.0,1,1 Campaign,1.0


In [19]:
# Contact frequency: contact_frequency_score

campaign_score.loc[campaign_score["contact_frequency"] <= 2,"contact_frequency_score"] = 1
campaign_score.loc[(campaign_score["contact_frequency"] > 2) & (campaign_score["contact_frequency"] <= 4),"contact_frequency_score"] = 2
campaign_score.loc[campaign_score["contact_frequency"] > 4,"contact_frequency_score"] = 3



# Duration of Contact: duration_of_contact_score

campaign_score.loc[campaign_score["sum_duration_of_contact"] <= 500,"duration_of_contact_score"] = 1
campaign_score.loc[(campaign_score["sum_duration_of_contact"] > 500) & (campaign_score["sum_duration_of_contact"] <= 1000),"duration_of_contact_score"] = 2
campaign_score.loc[campaign_score["sum_duration_of_contact"] > 1000,"duration_of_contact_score"] = 3


# Avg Duration of Contact: avg_duration_of_contact_score

campaign_score.loc[campaign_score["avg_duration_of_contact"] <= 200,"avg_duration_of_contact_score"] = 1
campaign_score.loc[(campaign_score["avg_duration_of_contact"] > 200) & (campaign_score["avg_duration_of_contact"] <= 400),"avg_duration_of_contact_score"] = 2
campaign_score.loc[campaign_score["avg_duration_of_contact"] > 400,"avg_duration_of_contact_score"] = 3


# Campaign Rate: campaign_rate_score

campaign_score.loc[(campaign_score["campaign_engagement_cnt"] < 5) & (campaign_score["campaign_rate_per_customer"] <= 0.5), "campaign_rate_score"] = 1
campaign_score.loc[(campaign_score["campaign_engagement_cnt"] < 5) & (campaign_score["campaign_rate_per_customer"] > 0.5) & (campaign_score["campaign_rate_per_customer"] <= 0.75), "campaign_rate_score"] = 2
campaign_score.loc[(campaign_score["campaign_engagement_cnt"] < 5) & (campaign_score["campaign_rate_per_customer"] > 0.75) , "campaign_rate_score"] = 3

campaign_score.loc[(campaign_score["campaign_engagement_cnt"] >= 5) & (campaign_score["campaign_rate_per_customer"] <= 0.25), "campaign_rate_score"] = 1
campaign_score.loc[(campaign_score["campaign_engagement_cnt"] >= 5) & (campaign_score["campaign_rate_per_customer"] > 0.25) & (campaign_score["campaign_rate_per_customer"] <= 0.5), "campaign_rate_score"] = 2
campaign_score.loc[(campaign_score["campaign_engagement_cnt"] >= 5) & (campaign_score["campaign_rate_per_customer"] > 0.5) , "campaign_rate_score"] = 3



# Campaign Engagement: campaign_engagement_score
campaign_score.loc[campaign_score["campaign_engagement_cnt"] == 1,"campaign_engagement_score"] = 1
campaign_score.loc[(campaign_score["campaign_engagement_cnt"] > 1) & (campaign_score["campaign_engagement_cnt"] <= 4),"campaign_engagement_score"] = 2
campaign_score.loc[campaign_score["campaign_engagement_cnt"] > 4,"campaign_engagement_score"] = 3


In [20]:
# Show the first 5 rows
campaign_score.head()

,customer_id,campaign,y,count_respond,contact_per_campaign,response_rate,contact_frequency,sum_duration_of_contact,avg_duration_of_contact,campaign_engagement_cnt,campaign_engagement_level,campaign_rate_per_customer,contact_frequency_score,duration_of_contact_score,avg_duration_of_contact_score,campaign_rate_score,campaign_engagement_score
0,17_student_single_basic.9y_unknown_unknown,2,yes,1,1,1.0,1,498,498.0,1,1 Campaign,1.0,1.0,1.0,3.0,3.0,1.0
1,17_student_single_unknown_no_yes,1,yes,1,1,1.0,1,896,896.0,1,1 Campaign,1.0,1.0,2.0,3.0,3.0,1.0
2,18_student_single_basic.4y_yes_no,1,yes,1,2,0.5,2,548,274.0,1,1 Campaign,0.5,1.0,2.0,2.0,1.0,1.0
3,18_student_single_basic.6y_no_yes,2,yes,1,1,1.0,1,368,368.0,1,1 Campaign,1.0,1.0,1.0,2.0,3.0,1.0
4,18_student_single_basic.9y_no_no,2,yes,1,1,1.0,1,412,412.0,1,1 Campaign,1.0,1.0,1.0,3.0,3.0,1.0


In [21]:
# Calculate the final score
campaign_score["overall_engagement_score"] = (campaign_score["contact_frequency_score"] * 8) +\
                                              (campaign_score["duration_of_contact_score"] * 7) +\
                                              (campaign_score["avg_duration_of_contact_score"] * 6) +\
                                              (campaign_score["campaign_engagement_score"] * 5) +\
                                              (campaign_score["campaign_rate_score"] * 9)

In [22]:
# Categorize based on threshold, save it into a new column called `engagement_category`

campaign_score.loc[campaign_score["overall_engagement_score"] <= 48,"engagement_category"] = "Low engagement"
campaign_score.loc[(campaign_score["overall_engagement_score"] > 48) & (campaign_score["overall_engagement_score"] <= 68),"engagement_category"] = "Moderately engaged"
campaign_score.loc[campaign_score["overall_engagement_score"] > 68,"engagement_category"] = "Highly engaged"

In [23]:
# Sanity check the result using `.value_counts()` function
campaign_score["engagement_category"].value_counts()

engagement_category
Highly engaged        2855
Moderately engaged     737
Low engagement         101
Name: count, dtype: int64

In [26]:

len(campaign_score)

3693